# 항공기 탑승객 만족도 데이터 분석

이 노트북은 항공기 탑승객 만족도 예측 데이터를 분석하고, GridSearch를 사용하여 모델을 최적화하는 과정을 담고 있습니다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/Air_Satisfaction.csv'
data = pd.read_csv(path)
data.dropna(inplace=True)
data['satisfaction'] = np.where(data['satisfaction'] =='satisfied', 1 ,0)

# 범주형 변수 가변수화
data = pd.get_dummies(data, drop_first=True)

target = 'satisfaction'
X = data.drop(target, axis=1)
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

data.head()

,id,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,...,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus
0,70172,13,460,3,4,3,1,5,3,5,...,5,5,25,18.0,0,True,False,True,False,True
1,5047,25,235,3,2,3,3,1,3,1,...,4,1,1,6.0,0,True,True,False,False,False
2,110028,26,1142,2,2,2,2,5,5,5,...,4,5,0,0.0,1,False,False,False,False,False
3,24026,25,562,2,5,5,5,2,2,2,...,4,2,11,9.0,0,False,False,False,False,False
4,119299,61,214,3,3,3,3,4,5,5,...,3,3,0,0.0,1,True,False,False,False,False


## (1) 의사결정 나무 (Decision Tree)

의사결정 나무는 데이터의 특징을 바탕으로 스무고개 하듯이 질문을 던져 정답을 찾아가는 모델입니다. 
너무 깊게 학습하면 과적합(Overfitting)이 발생할 수 있으므로 `max_depth`와 `min_samples_leaf`를 적절히 조절해야 합니다.

**튜닝할 하이퍼파라미터:**
- **max_depth** : 2 ~ 10 사이
- **min_samples_leaf** : 5 ~ 100 사이

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 1. 모델 정의
dt = DecisionTreeClassifier(random_state=42)

# 2. 하이퍼파라미터 그리드 설정
# max_depth: 2부터 10까지
# min_samples_leaf: 5부터 100까지 (간격을 5로 설정하여 탐색 효율화)
param_grid_dt = {
    'max_depth': range(2, 11),
    'min_samples_leaf': range(5, 101, 5)
}

# 3. GridSearchCV 설정
grid_dt = GridSearchCV(
    dt, 
    param_grid_dt, 
    cv=5, 
    scoring='accuracy', 
    n_jobs=-1
)

# 4. 학습 수행 (데이터 X_train, y_train이 로드되었다고 가정)
grid_dt.fit(X_train, y_train)

# 결과 출력
print("Best Params:", grid_dt.best_params_)
print("Best Score:", grid_dt.best_score_)

## (2) K-최근접 이웃 (KNN)

KNN(K-Nearest Neighbors)은 새로운 데이터가 들어왔을 때 가장 가까운 k개의 이웃을 보고 다수결로 분류하는 알고리즘입니다. 
이웃의 수(`n_neighbors`)와 거리를 재는 방식(`metric`)이 성능에 큰 영향을 미칩니다.

**튜닝할 하이퍼파라미터:**
- **n_neighbors** : 2 ~ 70 사이
- **metric** : euclidean, manhattan

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# 1. 모델 정의
knn = KNeighborsClassifier()

# 2. 하이퍼파라미터 그리드 설정
# n_neighbors: 2부터 70까지
# metric: 유클리드 거리와 맨해튼 거리 두 가지 방식 사용
param_grid_knn = {
    'n_neighbors': range(2, 71),
    'metric': ['euclidean', 'manhattan']
}

# 3. GridSearchCV 설정
grid_knn = GridSearchCV(
    knn, 
    param_grid_knn, 
    cv=5, 
    scoring='accuracy', 
    n_jobs=-1
)

# 4. 학습 수행
grid_knn.fit(X_train, y_train)

# 결과 출력
print("Best Params:", grid_knn.best_params_)
print("Best Score:", grid_knn.best_score_)

Best Params: {'metric': 'manhattan', 'n_neighbors': 29}
Best Score: 0.6446991404011462


## (3) 서포트 벡터 머신 (SVM)

SVM은 데이터를 가장 잘 나누는 경계선(초평면)을 찾는 강력한 모델입니다. 
데이터가 비선형일 경우 `kernel` 트릭을 사용하며, `C`와 `gamma`를 통해 모델의 복잡도를 조절합니다.

**튜닝할 하이퍼파라미터:**
- **gamma** : 0.1 ~ 10 사이
- **C** : 0.1 ~ 100 사이
- **kernel** : rbf, linear, poly

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# 1. 모델 정의
svm = SVC(random_state=42)

# 2. 하이퍼파라미터 그리드 설정
# C: 규제 강도 (낮을수록 규제가 강함)
# gamma: 결정 경계의 곡률 (높을수록 복잡함)
# kernel: 데이터를 고차원으로 매핑하는 함수
param_grid_svm = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.1, 1, 5, 10],
    'kernel': ['rbf', 'linear', 'poly']
}

# 3. GridSearchCV 설정
grid_svm = GridSearchCV(
    svm, 
    param_grid_svm, 
    cv=5, 
    scoring='accuracy', 
    n_jobs=-1
)

# 4. 학습 수행
grid_svm.fit(X_train, y_train)

# 결과 출력
print("Best Params:", grid_svm.best_params_)
print("Best Score:", grid_svm.best_score_)

KeyboardInterrupt: 